<a href="https://colab.research.google.com/github/pradigtaMD/UAS-KB/blob/main/Face_Recognition_Project_KB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from skimage.feature import hog
from tensorflow import keras
from tensorflow.keras import layers
import os
from sklearn.preprocessing import LabelEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Function to load images from a directory
def load_images_from_folder(folder):
    images = []
    labels = []
    for person in os.listdir(folder):
        person_folder = os.path.join(folder, person)
        if os.path.isdir(person_folder):
            for filename in os.listdir(person_folder):
                img = cv2.imread(os.path.join(person_folder, filename))
                if img is not None:
                    images.append(img)
                    labels.append(person)
    return np.array(images), np.array(labels)


In [ ]:
# Function to preprocess images
def preprocess_images(images):
    resized_images = [cv2.resize(img, (140, 140)) for img in images]
    return np.array(resized_images)



In [ ]:
# Function to extract HOG features from an image
def extract_hog_features(image):
    features, _ = hog(image, orientations=9, pixels_per_cell=(8, 8),
                      cells_per_block=(2, 2), visualize=True, multichannel=True)
    return features


In [ ]:
# Function to create a more complex CNN model
def create_complex_cnn(input_shape, num_classes):
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
        layers.Dropout(0.5),  # Add dropout for regularization
        layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
# Load face images and labels
folder_path = '/content/drive/MyDrive/Dataset'  # Replace with the path to your dataset
images, labels = load_images_from_folder(folder_path)

<ipython-input-3-09215b53db31>:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(images), np.array(labels)


In [ ]:
# Use LabelEncoder to convert string labels to numeric format
label_encoder = LabelEncoder()
numeric_labels = label_encoder.fit_transform(labels)

In [ ]:
# Preprocess images
rescaled_images = preprocess_images(images)

In [ ]:
# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(rescaled_images, numeric_labels, test_size=0.2, random_state=42)

In [ ]:
# Train the more complex CNN
input_shape = X_train[0].shape
num_classes = len(np.unique(numeric_labels))
complex_cnn_model = create_complex_cnn(input_shape, num_classes)
complex_cnn_model.fit(X_train, y_train, epochs=20)

Epoch 1/20
31/31 [==============================] - 41s 1s/step - loss: 169.3914 - accuracy: 0.0358
Epoch 2/20
31/31 [==============================] - 39s 1s/step - loss: 9.5334 - accuracy: 0.1411
Epoch 3/20
31/31 [==============================] - 40s 1s/step - loss: 8.9359 - accuracy: 0.3395
Epoch 4/20
31/31 [==============================] - 40s 1s/step - loss: 8.3615 - accuracy: 0.5573
Epoch 5/20
31/31 [==============================] - 39s 1s/step - loss: 7.8169 - accuracy: 0.6963
Epoch 6/20
31/31 [==============================] - 41s 1s/step - loss: 7.6722 - accuracy: 0.7904
Epoch 7/20
31/31 [==============================] - 39s 1s/step - loss: 7.1537 - accuracy: 0.8609
Epoch 8/20
31/31 [==============================] - 42s 1s/step - loss: 6.8860 - accuracy: 0.8793
Epoch 9/20
31/31 [==============================] - 39s 1s/step - loss: 7.0602 - accuracy: 0.8620
Epoch 10/20
31/31 [==============================] - 41s 1s/step - loss: 6.8269 - accuracy: 0.8732
Epoch 11/20
31/31

In [ ]:
# Extract feature maps from the convolutional neural network
train_feature_maps = complex_cnn_model.predict(X_train)
test_feature_maps = complex_cnn_model.predict(X_test)

8/8 [==============================] - 2s 234ms/step


In [ ]:
# Train a Gaussian KELM classifier
kelm_classifier = SVC(kernel='rbf')
kelm_classifier.fit(train_feature_maps, y_train)


SVC()

In [ ]:
# Testing
y_pred = kelm_classifier.predict(test_feature_maps)


In [ ]:
# Display some predictions
for i in range(10):  # Adjust the range as needed
    actual_label = label_encoder.inverse_transform([y_test[i]])[0]
    predicted_label = label_encoder.inverse_transform([y_pred[i]])[0]
    print(f"Actual: {actual_label}, Predicted: {predicted_label}")

Actual: Garima Panta, Predicted: Nischal Basnet
Actual: Sabin Shrestha, Predicted: Niraj Baral
Actual: Pooja Sharma, Predicted: Anuradha Koirala
Actual: Keki Adhikari, Predicted: Niraj Baral
Actual: Gauri Malla, Predicted: Pooja Sharma
Actual: Pooja Sharma, Predicted: Amrita Acharia
Actual: Aryan Sigdel, Predicted: Kushal Thapa
Actual: Priyanka Karki, Predicted: Barsha Raut
Actual: Deepak Raj Giri, Predicted: Rajesh Hamal
Actual: Niraj Baral, Predicted: Paras Khadka


In [ ]:
# Evaluate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 27.76%
